### My_UNETR++ (Synapese)

모델 구조  
(my_unetr_pp/training/network_training/unetr_pp_trainer_synapse.py)

In [1]:
import sys
sys.path.append('my_unetr_plus_plus')
 
import torch
import torch.nn as nn
from pytorch_model_summary import summary
from my_unetr_plus_plus.unetr_pp.network_architecture.synapse.unetr_pp_synapse import UNETR_PP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(device)

cuda


In [2]:
input_channels=1
num_classes=14
crop_size=[64, 128, 128]

전체 모델  
(encoder와 decoder에 nfce 모듈 적용됨)

In [3]:
# 네트워크
network = UNETR_PP(in_channels=input_channels,
                   out_channels=num_classes,
                   img_size=crop_size,
                   feature_size=16,
                   num_heads=4,
                   depths=[3, 3, 3, 3],
                   dims=[32, 64, 128, 256],
                   do_ds=True,)
network=network.to(device) # Network

# 모델 요약 및 테스트
input=torch.zeros(1, 1, 64, 128, 128).cuda() # Input Shape : [B, C, D, H, W]
print(summary(network, input, show_input=True)) # Forwarding Shape
output=network(input) 
print('output shape:') # Output Shape
for i,out in enumerate(output):
    print(i,':',out.shape)

---------------------------------------------------------------------------------------------------
       Layer (type)                                    Input Shape         Param #     Tr. Param #
   UnetrPPEncoder-1                           [1, 1, 64, 128, 128]      26,919,880      26,919,880
     UnetResBlock-2                           [1, 1, 64, 128, 128]           7,360           7,360
           My_TIF-3           [1, 128, 8, 8, 8], [1, 256, 4, 4, 4]       2,368,256       2,368,256
           My_TIF-4         [1, 64, 16, 16, 16], [1, 128, 8, 8, 8]         920,256         920,256
           My_TIF-5       [1, 32, 32, 32, 32], [1, 64, 16, 16, 16]       2,537,120       2,537,120
     UnetrUpBlock-6           [1, 256, 4, 4, 4], [1, 128, 8, 8, 8]       3,421,880       3,421,880
     UnetrUpBlock-7         [1, 128, 8, 8, 8], [1, 64, 16, 16, 16]       2,355,912       2,355,912
     UnetrUpBlock-8       [1, 64, 16, 16, 16], [1, 32, 32, 32, 32]       9,633,680       9,633,680
     Unet

모델 구조

In [4]:
network.parameters

<bound method Module.parameters of UNETR_PP(
  (unetr_pp_encoder): UnetrPPEncoder(
    (downsample_layers): ModuleList(
      (0): Sequential(
        (0): Convolution(
          (conv): Conv3d(1, 32, kernel_size=(2, 4, 4), stride=(2, 4, 4), bias=False)
        )
        (1): GroupNorm(1, 32, eps=1e-05, affine=True)
      )
      (1): Sequential(
        (0): My_PatchMerging(
          (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (reduction): Linear(in_features=256, out_features=64, bias=False)
        )
      )
      (2): Sequential(
        (0): My_PatchMerging(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (reduction): Linear(in_features=512, out_features=128, bias=False)
        )
      )
      (3): Sequential(
        (0): My_PatchMerging(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (reduction): Linear(in_features=1024, out_features=256, bias=False)
        )
      )
    )
    (my